In [1]:
from __future__ import absolute_import, print_function

# --- System ---
import os
import sys
import warnings

# --- Utility ---
import pandas as pd
import numpy as np
import math
import random
import logging
import pickle
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# --- Plot --
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# --- Pytorch ---
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn

from torch.utils.data import Dataset, DataLoader, TensorDataset
from tqdm import tqdm
from datetime import datetime
from torch.utils.data import random_split

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
root_dir = os.getcwd()

In [4]:
dataPath = "data/statistics-3.csv"
df = pd.read_csv(dataPath)
# columnList = df.columns

# Dropping columns that are not required at the moment
df = df.drop(columns=[ 'Unnamed: 0', 'UUID', 'HOSTNAME', 'ALIAS', 'TIMESTAMP',
                       'THROUGHPUT (Receiver)', 'LATENCY (min.)', 'LATENCY (max.)', 
                       'CONGESTION (Receiver)', 'BYTES (Receiver)'
                     ])

# Pre-processing
pacing = df['PACING'].values
for i, p in enumerate(pacing):
    v, _ = p.split("gbit")
    pacing[i] = int(v)

df['PACING'] = pacing
df['CONGESTION (Sender)'] = (df['CONGESTION (Sender)'] == 'cubic').astype(int) # Cubic = 1 & BBRV2 = 0

X = df[['THROUGHPUT (Sender)', 'LATENCY (mean)', 'RETRANSMITS', 'STREAMS', 'CONGESTION (Sender)']].values
y = df['PACING'].values
y = y.astype('int')

In [5]:
df.head(5)

,STREAMS,PACING,THROUGHPUT (Sender),LATENCY (mean),RETRANSMITS,CONGESTION (Sender)
0,1,1,1.630381e+09,30185.5,1535,1
1,1,1,1.659032e+09,30193.0,1344,1
2,1,1,9.887439e+08,31422.5,0,1
3,1,2,1.976052e+09,31420.5,0,1
4,1,3,2.962443e+09,31489.0,0,1


In [5]:
# # Standerdization
# std_scale = preprocessing.StandardScaler().fit(df[['THROUGHPUT (Sender)', 'LATENCY (mean)', 'RETRANSMITS', 'STREAMS', 'CONGESTION (Sender)']])
# df_std = std_scale.transform(df[['THROUGHPUT (Sender)', 'LATENCY (mean)', 'RETRANSMITS', 'STREAMS', 'CONGESTION (Sender)']])

# # Normalization
minmax_scale = preprocessing.MinMaxScaler().fit(df[['THROUGHPUT (Sender)', 'LATENCY (mean)', 'RETRANSMITS', 'STREAMS', 'CONGESTION (Sender)']])
df_minmax = minmax_scale.transform(df[['THROUGHPUT (Sender)', 'LATENCY (mean)', 'RETRANSMITS', 'STREAMS', 'CONGESTION (Sender)']])

final_df = pd.DataFrame(df_minmax, columns=['THROUGHPUT (Sender)', 'LATENCY (mean)', 'RETRANSMITS', 'STREAMS', 'CONGESTION (Sender)'])
final_df.head(5)

X = final_df[['THROUGHPUT (Sender)', 'LATENCY (mean)', 'RETRANSMITS', 'STREAMS', 'CONGESTION (Sender)']].values


In [12]:
EPOCH = 400
BATCH = 32
LEARNING_RATE = 0.001

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)
X_test  = torch.tensor(X_test)
y_test  = torch.tensor(y_test) 

class CustomTensorDataset(Dataset):
    """
    TensorDataset with support of transforms.
    """
    def __init__(self, tensors, transform=None):
        assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[0][index]

        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]

        return x, y

    def __len__(self):
        return self.tensors[0].size(0)

# Dataset w/o any tranformations
traindata   = CustomTensorDataset(tensors=(X_train, y_train), transform=None)
trainloader = torch.utils.data.DataLoader(traindata, batch_size=BATCH)

testdata    = CustomTensorDataset(tensors=(X_test, y_test), transform=None)
testloader = torch.utils.data.DataLoader(testdata, batch_size=BATCH)

print(len(traindata), len(testdata))


128 56


In [15]:
class PacingOptimizer(nn.Module):
    def __init__(self):
        super(PacingOptimizer, self).__init__()
        self.fc1 = torch.nn.Linear (5, 32)
        self.fc2 = torch.nn.Linear (32, 32)
        self.fc3 = torch.nn.Linear (32, 32)
        self.fc4 = torch.nn.Linear (32, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = PacingOptimizer()
print( f"====================\nTotal params: {len(list(model.parameters()))}\n====================" )
# print(model)

Total params: 8


In [16]:
CE = nn.CrossEntropyLoss()
MSE = nn.MSELoss(reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)
# optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
bestloss = 10

for epoch in range(EPOCH):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        xs, ys = data
        xs, ys = xs.float(), ys.float()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output = model(xs)
        # print(output, ys)
        
        # loss = CE(output, ys)
        loss = MSE(ys, output)
        
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    print(f"[{epoch+1}/{EPOCH}] loss: {running_loss/len(trainloader.dataset):.3f}")
    running_loss = 0.0

print('Finished Training')

correct, acc, total = 0, 0, 0
with torch.no_grad():
    for xs, ys in testloader:
        xs, ys = xs.float(), ys.long()

        output = model(xs)
        loss = MSE(ys, output)
        running_loss += loss.item()
        # total += ys.size(0)
        # pred = torch.max(output, 1)[1]
        # correct += (pred == ys).sum().item()
    # acc = (100 * correct / total)
# print(acc)

[1/400] loss: 0.692
[2/400] loss: 0.649
[3/400] loss: 0.577
[4/400] loss: 0.488
[5/400] loss: 0.385
[6/400] loss: 0.278
[7/400] loss: 0.193
[8/400] loss: 0.172
[9/400] loss: 0.193
[10/400] loss: 0.186
[11/400] loss: 0.168
[12/400] loss: 0.166
[13/400] loss: 0.169
[14/400] loss: 0.169
[15/400] loss: 0.167
[16/400] loss: 0.165
[17/400] loss: 0.165
[18/400] loss: 0.165
[19/400] loss: 0.165
[20/400] loss: 0.164
[21/400] loss: 0.164
[22/400] loss: 0.164
[23/400] loss: 0.164
[24/400] loss: 0.164
[25/400] loss: 0.164
[26/400] loss: 0.164
[27/400] loss: 0.164
[28/400] loss: 0.164
[29/400] loss: 0.164
[30/400] loss: 0.164
[31/400] loss: 0.164
[32/400] loss: 0.164
[33/400] loss: 0.164
[34/400] loss: 0.164
[35/400] loss: 0.164
[36/400] loss: 0.164
[37/400] loss: 0.164
[38/400] loss: 0.164
[39/400] loss: 0.164
[40/400] loss: 0.164
[41/400] loss: 0.164
[42/400] loss: 0.164
[43/400] loss: 0.164
[44/400] loss: 0.164
[45/400] loss: 0.164
[46/400] loss: 0.164
[47/400] loss: 0.164
[48/400] loss: 0.164
[

In [ ]:
# import os
# import sys

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import warnings

# %matplotlib inline

# # --- Sklearn ---
# from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
# from sklearn import decomposition, discriminant_analysis
# from sklearn.model_selection import GridSearchCV

# # --- Models ---
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn import svm
# from sklearn import neural_network
# from sklearn.linear_model import LogisticRegression

# from sklearn.preprocessing import LabelEncoder
# from sklearn.cross_validation import train_test_split

# # --- Utility ---
# import os
# import pickle, torch
# import numpy as np, pandas as pd
# import seaborn as sn
# import matplotlib.pyplot as plt


# dataPath = "data/statistics (pacing).csv"
# df_full = pd.read_csv(dataPath)
# print(df_full.describe())

In [ ]:
columnList = df_full.columns
print(f"Total columns list: {columnList}")

# Dropping columns that are not required at the moment
df = df_full.drop(columns=[ 'Unnamed: 0', 'UUID', 'HOSTNAME', 'ALIAS', 'TIMESTAMP', 'STREAMS',
                            'THROUGHPUT (Receiver)', 'LATENCY (min.)', 'LATENCY (max.)', 
                            'CONGESTION (Receiver)', 'BYTES (Receiver)'
                          ])

print(f"New columns list: {df.columns}")

In [ ]:
df.describe()

In [ ]:
# df.head(5)

# Preprocessing

In [ ]:
pacing = df['PACING'].values
for i, p in enumerate(pacing):
    v, _ = p.split("gbit")
    pacing[i] = int(v)

df['PACING'] = pacing
# df['CONGESTION (Sender)'] = df['CONGESTION (Sender)']=="cubic"=1
# df['CONGESTION (Sender)'] = df['CONGESTION (Sender)']=="bbr2"=0
df['CONGESTION (Sender)'] = (df['CONGESTION (Sender)'] == 'cubic').astype(int)

In [ ]:

df.head(5)

In [ ]:
# sns.set(style='whitegrid', context='notebook')
# cols = ['THROUGHPUT (Sender)', 'LATENCY (mean)', 'RETRANSMITS', 'CONGESTION (Sender)', 'PACING']

# sns.pairplot(df[cols], height=3)
# plt.tight_layout()
# # plt.savefig('./figures/scatter.png', dpi=300)
# plt.show()

# Dataset

In [ ]:
X = df[['THROUGHPUT (Sender)', 'LATENCY (mean)', 'RETRANSMITS']].values
y = df['PACING'].values
y = y.astype('int')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
def train_and_tune(X, y, model, parameters, scoring='f1_macro', kfold=5, verbose=0):
    """
    X:          array-like of shape (n_samples, n_features)
    y:          array-like of shape (n_samples,)
    model:      (object) a sklearn model class
    parameters: (dict) contains the parameters you want to tune in the model
    metric:     (str) the metric used to evaluate the quality of the model
    return:     a trained model with the best parameters
    """
    cvSearchObj = GridSearchCV(model,
                               parameters,
                               scoring=scoring,
                               n_jobs=-1,
                               cv=kfold,
                               verbose=verbose)
    cvSearchObj.fit(X,y)
    return cvSearchObj.best_estimator_

def save_model(filename, model):
    """
    filename: Filename to save the model
    model:    Model weights to be saved
    """
    pickle.dump(model, open(filename, 'wb'))
    print("Model Saved")

def load_model(filename):
    """
    filename: Filename to load the model
    return:   Model weights that are reloaded
    """
    model_reloaded = pickle.load(open(filename, 'rb'))
    return model_reloaded


def MLP(train, save, test):
    filename = "./mlpBest.pkl"
    mlp = neural_network.MLPClassifier(random_state=999)
    if train:
        '''
        Train
        '''
        params = {"alpha" : [0.0001],
                "learning_rate_init" : [0.001],
                "batch_size" : [32, 64, 128],
                "activation" : ["relu"],
                "early_stopping" : [True],
                "hidden_layer_sizes" : [10, 50, 100],
                }

        mlpBest = train_and_tune(X, y,
                                 mlp,
                                 params,
                                 scoring='f1_macro',
                                 kfold=5)

        if save:
            save_model(filename, mlpBest)

    if test:
        '''
        Test
        '''
        mlpBest_reloaded = load_model(filename)
        pred = mlpBest_reloaded.predict(X)
        acc  = mlpBest_reloaded.score(X, y)
        
        # cf_matrix = confusion_matrix(y, pred)
        # df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix) *10, index = [i for i in classes],
        #                      columns = [i for i in classes])
        # plt.figure(figsize = (12,10))
        # sn.heatmap(df_cm, annot=True)
        
        print("Accuracy: ", acc)

MLP(train=True, save=True, test=True)

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)
# le.transform(['M', 'B'])

model = LogisticRegression()
model.fit(X_train, y_train)
print('Test Accuracy: %.3f' % model.score(X_test, y_test))